In [1]:
import pandas as pd

In [6]:
inspections = pd.read_csv("../data/inspections.csv", index_col=0)

In [7]:
inspections.head(3)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,40647906,TERRACE ON THE PARK,QUEENS,5211,111 STREET,11368,7185925000,American,04/07/2016,Violations were cited in the following area(s).,10B,Plumbing not properly installed or maintained;...,Not Critical,25.0,NaN,NaN,09/24/2016,Cycle Inspection / Initial Inspection
1,41396241,KITCHEN PROVANCE,MANHATTAN,55,WEST 38 STREET,10018,2128692561,American,05/24/2013,Violations were cited in the following area(s).,10H,Proper sanitization not provided for utensil w...,Not Critical,26.0,NaN,NaN,09/24/2016,Cycle Inspection / Initial Inspection
2,50009921,HESTIA MARKETPLACE,MANHATTAN,513,FASHION AVE,10018,2127683838,American,06/03/2016,Violations were cited in the following area(s).,10B,Plumbing not properly installed or maintained;...,Not Critical,24.0,NaN,NaN,09/24/2016,Cycle Inspection / Initial Inspection


In [29]:
inspections['INSPECTION DATE'] = pd.to_datetime(inspections['INSPECTION DATE'])

In [47]:
# random_camis = inspections.sample().iloc[0]['CAMIS']
# inspections[inspections['CAMIS'] == random_camis].sort_values(by='INSPECTION DATE')['INSPECTION DATE']

Get initial inspection date.

In [78]:
inspections_f = inspections.groupby('CAMIS')\
                            .apply(lambda df: inspections[inspections['CAMIS'] == df['CAMIS'].iloc[0]]\
                                               .sort_values(by="INSPECTION DATE")\
                                               .iloc[0]\
                                               .drop('CAMIS')
                                  )\
                            .rename(columns={'INSPECTION DATE': 'INITIAL INSPECTION DATE'})

In [58]:
inspections_f.sample(1)

,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INITIAL INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
CAMIS,,,,,,,,,,,,,,,,,
50006167,NaN,MANHATTAN,31.0,OLIVER ST,10038,2.127663e+09,Chinese/Japanese,2014-04-21,Violations were cited in the following area(s).,08A,Facility not vermin proof. Harborage or condit...,Not Critical,29.0,NaN,NaN,09/24/2016,Pre-permit (Operational) / Initial Inspection


Not sure why, but this operation drops the `DBA` field. None others are affected. Maybe something with the indexing?

Let's take the most recent inspection date.

In [84]:
inspections_ff = inspections_f.copy()
inspections_ff['LATEST INSPECTION DATE'] = inspections_ff\
        .apply(lambda srs: inspections[inspections['CAMIS'] == srs.name]\
                                  .sort_values(by='INSPECTION DATE')\
                                  .iloc[-1]\
                                  ['INSPECTION DATE'],
              axis='columns')

Rename the `INSPECTION TYPE` to `INITIAL INSPECTION TYPE` (this flags new establishments from pre-existing ones, in the cotext of our dataset).

In [89]:
inspections_ff = inspections_ff.rename(columns={'INSPECTION TYPE': 'INITIAL INSPECTION TYPE'})

Checking the flags:

In [91]:
inspections_ff['INITIAL INSPECTION TYPE'].value_counts()

Cycle Inspection / Initial Inspection                          12537
Pre-permit (Operational) / Initial Inspection                   7630
Pre-permit (Non-operational) / Initial Inspection               2256
Cycle Inspection / Re-inspection                                 877
Administrative Miscellaneous / Initial Inspection                683
Smoke-Free Air Act / Initial Inspection                          281
Trans Fat / Initial Inspection                                   247
Pre-permit (Operational) / Re-inspection                          82
Trans Fat / Compliance Inspection                                 80
Trans Fat / Re-inspection                                         75
Inter-Agency Task Force / Initial Inspection                      53
Calorie Posting / Initial Inspection                              30
Trans Fat / Second Compliance Inspection                          27
Administrative Miscellaneous / Re-inspection                      27
Smoke-Free Air Act / Re-inspection

A lot of them are null.

In [93]:
inspections_ff['INITIAL INSPECTION TYPE'].isnull().astype(int).sum()

1094

Remember, this is new establishments that have not been inspected yet. That the numbers match up here is encouraging.

In [94]:
inspections['INSPECTION TYPE'].isnull().astype(int).sum()

1094

We reattach the lost DBA column.

In [95]:
inspections_ff.head(1)

,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INITIAL INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INITIAL INSPECTION TYPE,LATEST INSPECTION DATE
CAMIS,,,,,,,,,,,,,,,,,,
30075445,NaN,BRONX,1007.0,MORRIS PARK AVE,10462,7.188925e+09,Bakery,2013-06-01,Violations were cited in the following area(s).,16B,The original nutritional fact labels and/or in...,Not Critical,NaN,NaN,NaN,09/24/2016,Trans Fat / Compliance Inspection,2016-02-18


In [97]:
inspections_fff = inspections_ff.copy()
inspections_fff['DBA'] = inspections_fff.apply(lambda srs: 
                                                   inspections[inspections['CAMIS'] == srs.name]\
                                                   .iloc[0]['DBA'],
                                               axis='columns')

Prepend descriptors, to more easily distinguish things down the road.

In [104]:
inspections_fff.columns = ["DOHMH " + col for col in inspections_fff.columns]

In [105]:
inspections_fff.sample()

,DOHMH DBA,DOHMH BORO,DOHMH BUILDING,DOHMH STREET,DOHMH ZIPCODE,DOHMH PHONE,DOHMH CUISINE DESCRIPTION,DOHMH INITIAL INSPECTION DATE,DOHMH ACTION,DOHMH VIOLATION CODE,DOHMH VIOLATION DESCRIPTION,DOHMH CRITICAL FLAG,DOHMH SCORE,DOHMH GRADE,DOHMH GRADE DATE,DOHMH RECORD DATE,DOHMH INITIAL INSPECTION TYPE,DOHMH LATEST INSPECTION DATE
CAMIS,,,,,,,,,,,,,,,,,,
40374834,CASA BELLA,MANHATTAN,127.0,MULBERRY STREET,10013,2.124314e+09,Italian,2013-08-15,Violations were cited in the following area(s).,06D,"Food contact surface not properly washed, rins...",Critical,6.0,NaN,NaN,09/24/2016,Cycle Inspection / Initial Inspection,2016-04-26


Now we get information from Yelp!

In [106]:
from yelp.client import Client
from yelp.oauth1_authenticator import Oauth1Authenticator
from yelp.errors import BusinessUnavailable
import os
import json

def import_credentials(filename='../data/yelp_credentials.json'):
    try:
        data = json.load(open(filename))
        return data
    except:
        raise IOError('This API requires Yelp credentials to work. Did you forget to define them?')

credentials = import_credentials()

auth = Oauth1Authenticator(
    consumer_key=credentials['consumer_key'],
    consumer_secret=credentials['consumer_secret'],
    token=credentials['token'],
    token_secret=credentials['token_secret']
)

client = Client(auth)

In [107]:
from tqdm import tqdm

In [137]:
def yelp_phone_fetch(num):
    """
    Performs the phone search described in notebook 02 to fetch information on the entity associated with a number.
    """
    if not num:
        return None
    else:
        try:
            business = client.phone_search(num).businesses[0]
            return {
                'Yelp ID': business.id,
                'Yelp Is Claimed': business.is_claimed,
                'Yelp Is Closed': business.is_closed,
                'Yelp Name': business.name,
                'Yelp URL': business.url,
                'Yelp Review Count': business.review_count,
                'Yelp Categories': business.categories,
                'Yelp Rating': business.rating,
                'Yelp Address': business.location.address,
                'Yelp Neighborhoods': business.location.neighborhoods,
                'Yelp Latitude': business.location.coordinate.latitude,
                'Yelp Longitude': business.location.coordinate.longitude,
                   }
        except IndexError:  # Phone search failed!
            return None

After some testing there appears to be a *very* significant miss rate. Remember, we're fuzzy matching phone numbers from one data set with phone numbers in another dataset. It's far from 100% that we'll get something back out. I'm interested in what percentage of the time we're successful.

In [129]:
def random_number_dba_tuple():
    random_number, random_dba = inspections_fff.sample().iloc[0][['DOHMH PHONE', 'DOHMH DBA']]
    random_number = int(random_number)
    return random_number, random_dba

In [130]:
random_number_dba_tuple()

(2126949293, 'POPEYES LOUISIANA KITCHEN')

In [127]:
yelp_phone_fetch(random_number_dba_tuple())

{'Yelp Address': ['41 E 11th St'],
 'Yelp Categories': [Category(name='Japanese', alias='japanese')],
 'Yelp ID': 'ootoya-greenwich-village-new-york',
 'Yelp Is Claimed': True,
 'Yelp Is Closed': False,
 'Yelp Latitude': 40.7333107,
 'Yelp Longitude': -73.9929962,
 'Yelp Name': 'Ootoya Greenwich Village',
 'Yelp Neighborhoods': ['Greenwich Village'],
 'Yelp Rating': 4.0,
 'Yelp Review Count': 135,
 'Yelp URL': 'https://www.yelp.com/biz/ootoya-greenwich-village-new-york?adjust_creative=dkJPGu_jtTyHwsEgZIZN6g&utm_campaign=yelp_api&utm_medium=api_v2_phone_search&utm_source=dkJPGu_jtTyHwsEgZIZN6g'}

Hey I've been there! Nice. More rigorously:

In [133]:
one_hundred_randoms = [random_number_dba_tuple() for i in range(100)]

In [138]:
testset = [yelp_phone_fetch(num) for num, placename in tqdm(one_hundred_randoms)]


100%|████████████████████████████████████████| 100/100 [00:27<00:00,  4.71it/s]


In [145]:
np.array([entity == None for entity in testset]).astype(int).sum()

19

19 misses in 100 randoms. So we're able to match 80% of the time. Not bad as far as fuzzy matches go.

If our data is missing at random (MAR), then we are happy, because this makes for a statistically valid sample of all restaurants in New York City&mdash;we can simply drop the other values.

But there's a high likelihood, in my professional opinion, that the data is missing not at a random (MNAR). I expect that it's more likely for a restaurant or eatery in a *poor* neighborhood to be missing the phone number information in Yelp! than one in a richer, whiter community. This is a severe under-reporting issue that will invalidate any conclusions we try to make using this data with regarding to "gentrification" and whatnot.

We'll need to validate the data geospatially. We'll do that next.

Run the full Yelp! API data through. A time estimate:

In [150]:
len(inspections_fff)

26074

In [149]:
str((100/27)*len(inspections_fff)) + " seconds"

'96570.37037037036 seconds'

Oy vey.

More importantly, this is slightly more than the 25000 daily call limit Yelp! imposes. So we'll have to split this op over two days!

Arguably we don't really need to get all 25000 data items. But there's really no reason to skimp now.

In [154]:
from tqdm import tqdm_notebook

In [159]:
raw_yelp_5000 = [yelp_phone_fetch(int(num)) if pd.notnull(num) else None for num in tqdm_notebook(inspections_fff['DOHMH PHONE'][:5000])]

AttributeError: 'NoneType' object has no attribute 'latitude'

In [146]:
inspections_fff.to_csv("../data/inspections_flattened_initial.csv", encoding="utf-8")